In [1]:
import json
import evaluate

In [2]:
evaluate.load("rouge")

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

## NAIVE
1. input: document => output: summary
    -  rouge 다시 측정할 필요 없음

In [3]:
with open(f'../results/NAIVE/results_agg_temp_new.json') as f:
    data_NAIVE = json.load(f)

In [4]:
data_NAIVE['prediction'][0]['pred']

'Tom Brady, 37, was invited to the White House on Thursday.\nPresident Barack Obama says he was worried that 11 out of 12 of 12 footballs were under inflated.\nHe also joked that he wished Chicago could learn something from Boston\'s championship win.\nThe Super Bowl MVP and the team\'s biggest star - Rob Gronkowski, 37 - were in town for the ceremony.\nBut he didn\'t attend the ceremony.The Superbowl MVP and star star - Tom Brady, 36, didn\'t get to attend the event.\nTom Brady was pictured with President Barack Obama at the White house.\nBr Brady was also pictured with the President in 2005 and rival Donovan McNabb.\nObama also received a custom jersey and a custom football from the team.\nPatriots star - Brady, 41, didn’t attend the Whitehouse ceremony.\ninc "\xa0 -Key Sentence:'

In [5]:
data_NAIVE['prediction'][0]['label']

"Brady cited 'prior family commitments' in bowing out of meeting with Obama.\nHas been to the White House to meet President George W. Bush for previous Super Bowl wins."

In [6]:
NAIVE_SUM_rouge = data_NAIVE['metric']['rouge']
NAIVE_SUM_rouge

{'rouge1': 0.29766689973067506,
 'rouge2': 0.11573642880594928,
 'rougeL': 0.19151852364248945,
 'rougeLsum': 0.28249732233651165}

## KW
2. input: document => output: key words, summary

In [7]:
with open(f'../results/KW/results_agg_temp_new.json') as f:
    data_KW = json.load(f)

In [8]:
data_KW['prediction'][0]['pred'].replace('Key Words: ', '').split("\nSummary: ")

['sports commentators,sports commentators,football,sports administration,sports media,sports broadcasting,sports television,sports psychology,sports management,sports medicine,sports social media,social media,football administration,social networking,sports communication,sports marketing,sports personnel,sports fan,sports news,sports psychologist,sports commentator,sports analyst,sports journalist,sports pundits,sports commentary,sports,sports journalists,sports reporters,sports analysts,sports fans,sports officials,sports information,sports gossip,sports science,sports analysis,sports opinion,sports policy,sports research,sports education,sports behavior,sports relations,sports economics,sports law,sports communications,sports networking,social behavior,social psychology,social sciences,sports industry,sports political,sports',
 "Tom Brady received custom jersey and a custom football from the team at the White House on Thursday.\nHe's not going to be there this year.\nThe New England 

In [9]:
data_KW['prediction'][0]['label'].replace('Key Words: ', '').split("\nSummary: ")

['social media,sports medicine,public health,physiology,biomedical research,bioinformatics,cardiology,neurology,neonatology,nephrology',
 "Brady cited 'prior family commitments' in bowing out of meeting with Obama.\nHas been to the White House to meet President George W. Bush for previous Super Bowl wins."]

In [10]:
KW_KW_pred = []
KW_KW_label = []
KW_SUM_pred = []
KW_SUM_label = []
KW_no_summary = []
KW_no_summary_error = []

data_length = len(data_KW['prediction'])

for i in range(data_length):
    ex_pred = data_KW['prediction'][i]['pred'].replace('Key Words: ', '').split("\nSummary: ")
    ex_label = data_KW['prediction'][i]['label'].replace('Key Words: ', '').split("\nSummary: ")
    try:
        KW_KW_pred.append(ex_pred[0])
        KW_KW_label.append(ex_label[0])
        KW_SUM_pred.append(ex_pred[1]) 
        KW_SUM_label.append(ex_label[1])
    except:
        try:
            KW_KW_pred.append(ex_pred[0])
            KW_KW_label.append(ex_label[0])
            KW_no_summary.append(i)
            # print(f'Prediction {i} Has No Summary.')
        except: 
            KW_no_summary.append(i)
            KW_no_summary_error.append(i)
            print(f'Prediction {i} Has No Summary & Key Words Error Occured.')

In [11]:
KW_pred_length = len(data_KW['prediction'])
KW_no_summary_length = len(KW_no_summary)
print(f'{KW_pred_length}개 데이터 중 {KW_no_summary_length}개 데이터에 요약문이 없습니다.')

500개 데이터 중 146개 데이터에 요약문이 없습니다.


In [12]:
len(KW_KW_pred) == len(KW_KW_label) and len(KW_SUM_pred) == len(KW_SUM_label)

True

In [13]:
rouge = evaluate.load('rouge')

In [14]:
KW_KW_rouge = rouge.compute(references=KW_KW_label, predictions=KW_KW_pred, use_aggregator=True)
KW_SUM_rouge = rouge.compute(references=KW_SUM_label, predictions=KW_SUM_pred, use_aggregator=True)

In [15]:
KW_KW_rouge

{'rouge1': 0.13051620429624497,
 'rouge2': 0.055326316655400984,
 'rougeL': 0.10447204026591655,
 'rougeLsum': 0.10876921069623696}

In [16]:
KW_SUM_rouge

{'rouge1': 0.26312032955844566,
 'rouge2': 0.09489284290787497,
 'rougeL': 0.18729831906349462,
 'rougeLsum': 0.24550211010993514}

In [17]:
KW_KW_rouge_ex = rouge.compute(references=KW_KW_label, predictions=KW_KW_pred, use_aggregator=False)
KW_SUM_rouge_ex = rouge.compute(references=KW_SUM_label, predictions=KW_SUM_pred, use_aggregator=False)

## KS
3. input: document => output: key sentenece, summary

In [18]:
with open(f'../results/KS/results_agg_temp_new.json') as f:
    data_KS = json.load(f)

In [19]:
data_KS['prediction'][0]['pred'].replace('Key Sentence: ', '').split("\nSummary: ")

["President Barack Obama invited the Super Bowl champion New England Patriots to the White House on Thursday - but couldn't help but get one last deflategate joke in.",
 "The president opened his speech on the South Lawn by remarking 'that whole (deflgate) story got blown out of proportion,' referring to an investigation that 11 out of 12 footballs used in the AFC Championship game were under-inflated."]

In [20]:
data_KS['prediction'][0]['label'].replace('Key Sentence: ', '').split("\nSummary: ")

['Brady went to the White House to meet President George W Bush after winning the Super Bowl in 2005 and in 2004.',
 "Brady cited 'prior family commitments' in bowing out of meeting with Obama.\nHas been to the White House to meet President George W. Bush for previous Super Bowl wins."]

In [21]:
KS_KS_pred = []
KS_KS_label = []
KS_SUM_pred = []
KS_SUM_label = []
KS_no_summary = []
KS_no_summary_error = []

data_length = len(data_KS['prediction'])

for i in range(data_length):
    ex_pred = data_KS['prediction'][i]['pred'].replace('Key Sentence: ', '').split("\nSummary: ")
    ex_label = data_KS['prediction'][i]['label'].replace('Key Sentence: ', '').split("\nSummary: ")
    try:
        KS_KS_pred.append(ex_pred[0])
        KS_KS_label.append(ex_label[0])
        KS_SUM_pred.append(ex_pred[1]) 
        KS_SUM_label.append(ex_label[1])
    except:
        try :
            KS_KS_pred.append(ex_pred[0])
            KS_KS_label.append(ex_label[0])
            KS_no_summary.append(i)
            # print(f'Prediction {i} Has No Summary.')
        except: 
            KS_no_summary.append(i)
            KS_no_summary_error.append(i)
            print(f'Prediction {i} Has No Summary & Key Sentence Error Occured.')

In [22]:
KS_pred_length = len(data_KS['prediction'])
KS_no_summary_length = len(KS_no_summary)
print(f'{KS_pred_length}개 데이터 중 {KS_no_summary_length}개 데이터에 요약문이 없습니다.')

500개 데이터 중 174개 데이터에 요약문이 없습니다.


In [23]:
len(KS_KS_pred) == len(KS_KS_label) and len(KS_SUM_pred) == len(KS_SUM_label)

True

In [24]:
rouge = evaluate.load('rouge')

In [25]:
KS_KS_rouge = rouge.compute(references=KS_KS_label, predictions=KS_KS_pred, use_aggregator=True)
KS_SUM_rouge = rouge.compute(references=KS_SUM_label, predictions=KS_SUM_pred, use_aggregator=True)

In [26]:
KS_KS_rouge

{'rouge1': 0.3460665976893965,
 'rouge2': 0.2272930955662881,
 'rougeL': 0.30638699468046515,
 'rougeLsum': 0.30893421284970307}

In [27]:
KS_SUM_rouge

{'rouge1': 0.3009008074606278,
 'rouge2': 0.11779500085339398,
 'rougeL': 0.22062751833798439,
 'rougeLsum': 0.2661031298347436}

In [28]:
KS_KS_rouge_ex = rouge.compute(references=KS_KS_label, predictions=KS_KS_pred, use_aggregator=False)
KS_SUM_rouge_ex = rouge.compute(references=KS_SUM_label, predictions=KS_SUM_pred, use_aggregator=False)

## KWKS
4. input: document => ouptut: key words, key sentence, summary

In [29]:
with open(f'../results/KWKS/results_agg_temp_new.json') as f:
    data_KWKS = json.load(f)

In [30]:
temp = data_KWKS['prediction'][0]['pred'].replace('Key Words: ', '')
KW_KS_border = temp.find('\nKey Sentence: ')
KS_SUM_border = temp.find('\nSummary: ')
print(temp[:KW_KS_border].strip(),'\n', 
      temp[KW_KS_border+len('\nKey Sentence: '):KS_SUM_border].strip(), '\n', 
      temp[KS_SUM_border+len('\nSummary: '):].strip())

social media,sports medicine,public health,emergency medical services,human factors,ergonomics,occupational safety,suicide prevention,injury prevention,sports communication,sports 
 The president opened his speech on the South Lawn by remarking 'that whole (deflategate) story got blown out of proportion,' referring to an investigation that 11 out of 12 footballs used in the AFC Championship game were under-inflated. 
 President Barack Obama hails the high-profile Super Bowl celebrity celebrities.
He also hit out at former Massachusetts Senator and New England Patriots owner Bob Kraft.
Obama also hits out at Bill Belichick for giving the thumbs down.


In [31]:
KWKS_KW_pred = []
KWKS_KW_label = []
KWKS_KS_pred = []
KWKS_KS_label = []
KWKS_SUM_pred = []
KWKS_SUM_label = []
KWKS_no_summary = []

data_length = len(data_KWKS['prediction'])

for i in range(data_length):
    ex_pred = data_KWKS['prediction'][i]['pred'].replace('Key Words: ', '')
    ex_label = data_KWKS['prediction'][i]['label'].replace('Key Words: ', '')
    
    KW_KS_border_pred = ex_pred.find('\nKey Sentence: ')
    KS_SUM_border_pred = ex_pred.find('\nSummary: ')

    KW_KS_border_label = ex_label.find('\nKey Sentence: ')
    KS_SUM_border_label = ex_label.find('\nSummary: ')
    
    KWKS_KW_pred.append(ex_pred[:KW_KS_border_pred].strip())
    KWKS_KW_label.append(ex_label[:KW_KS_border_label].strip())

    KWKS_KS_pred.append(ex_pred[KW_KS_border_pred+len('\nKey Sentence: '):KS_SUM_border_pred].strip())
    KWKS_KS_label.append(ex_label[KW_KS_border_label+len('\nKey Sentence: '):KS_SUM_border_label].strip())

    # KWKS_SUM_pred.append(ex_pred[KS_SUM_border_pred+len('\nSummary: '):].strip())
    # KWKS_SUM_label.append(ex_label[KS_SUM_border_label+len('\nSummary: '):].strip())
    if "\nSummary" in ex_pred:
        KWKS_SUM_label.append(ex_label.split("\nSummary: ")[-1])
        KWKS_SUM_pred.append(ex_pred.split("\nSummary: ")[-1])

In [32]:
for summary  in KWKS_SUM_label[:10]:
    print(summary)

Brady cited 'prior family commitments' in bowing out of meeting with Obama.
Has been to the White House to meet
Reports emerged on social media  suggesting Mike Ashley was now the official owner
terrorism,human factors,occupational safety,injury prevention,ergonomics,terrorism,suicide prevention,humanitarian aid,terrorism.human factors.terrorism. al-Qaeda.
Key Sentence: A federal grand jury in California indicted Gadahn in 2006 for treason and material support to al Qaeda, charges related to his alleged involvement in terrorist activities that included "providing aid and comfort" and other services to al Qaeda, the FBI said.
Summary:
John Barnes appeared as a guest on Sky's A League Of Their Own show.
The 51-year-old rapped 1979
John Higgins beats Robert Milkins 10-5 to reach second round.
Higgins says he would love to challenge for World title again.
But the 39-year-old admits there are many more
Arsenal face Aston Villa in the FA Cup final at Wembley
Coutinho hit the only goal of the

In [33]:
for summary  in KWKS_SUM_pred[:10]:
    print(summary)

President Barack Obama hails the high-profile Super Bowl celebrity celebrities.
He also hit out at former Massachusetts Senator and New England Patriots owner Bob Kraft.
Obama also hits out at Bill Belichick for giving the thumbs down.
Reports emerged on social media suggest Mike Ashley has grabbed control of the club's badges.
Reports emerged that the Newcastle  owner was now the official owner of the Light Blues trademark.
Rangers have promised to conduct a speedy investigation into the claims.
NEW: Adam Gadahn, 36, was killed in January by al Qaeda in Pakistan.
He was the prime suspect in the 9/11 attacks, White House says.
His father says Gadahn was one of its most prominent members.
John Barnes and Jamie Redknapp performed Rapper's Delight - a spoof sketch called Britain's Got Sporting Talent.
The pair were joined by backing dancers and were 'dressed in a black hoodie and jeans'
Redknapp was equally adept on the mic as the pair entertained the audience.
The John Higgins took the B

In [34]:
len(KWKS_KW_pred) == len(KWKS_KW_label) and len(KWKS_KS_pred) == len(KWKS_KS_label) and len(KWKS_SUM_pred) == len(KWKS_SUM_label)

True

In [35]:
rouge = evaluate.load('rouge')

In [36]:
KWKS_KW_rouge = rouge.compute(references=KWKS_KW_label, predictions=KWKS_KW_pred, use_aggregator=True)
KWKS_KS_rouge = rouge.compute(references=KWKS_KS_label, predictions=KWKS_KS_pred, use_aggregator=True)
KWKS_SUM_rouge = rouge.compute(references=KWKS_SUM_label, predictions=KWKS_SUM_pred, use_aggregator=True)

In [40]:
KWKS_KW_rouge_ex = rouge.compute(references=KWKS_KW_label, predictions=KWKS_KW_pred, use_aggregator=False)
KWKS_KS_rouge_ex = rouge.compute(references=KWKS_KS_label, predictions=KWKS_KS_pred, use_aggregator=False)
KWKS_SUM_rouge_ex = rouge.compute(references=KWKS_SUM_label, predictions=KWKS_SUM_pred, use_aggregator=False)

## Evaluation Result

In [41]:
import pandas as pd

In [42]:
pd.DataFrame({'NAIVE_SUM' : NAIVE_SUM_rouge,
             'KW_KW' : KW_KW_rouge, 
             'KW_SUM' : KW_SUM_rouge, 
             'KS_KS' : KS_KS_rouge,
             'KS_SUM' : KS_SUM_rouge, 
             'KWKS_KW' : KWKS_KW_rouge, 
             'KWKS_KS' : KWKS_KS_rouge, 
             'KWKS_SUM' : KWKS_SUM_rouge})

NAIVE_SUM     KW_KW    KW_SUM     KS_KS    KS_SUM   KWKS_KW  \
rouge1      0.297667  0.130516  0.263120  0.346067  0.300901  0.345929   
rouge2      0.115736  0.055326  0.094893  0.227293  0.117795  0.205009   
rougeL      0.191519  0.104472  0.187298  0.306387  0.220628  0.284733   
rougeLsum   0.282497  0.108769  0.245502  0.308934  0.266103  0.284677   

            KWKS_KS  KWKS_SUM  
rouge1     0.280373  0.210458  
rouge2     0.146518  0.078380  
rougeL     0.238454  0.161458  
rougeLsum  0.237766  0.194101

In [43]:
pd.DataFrame({'NAIVE_SUM' : NAIVE_SUM_rouge,
             'KW_SUM' : KW_SUM_rouge, 
             'KS_SUM' : KS_SUM_rouge, 
             'KWKS_SUM' : KWKS_SUM_rouge})

NAIVE_SUM    KW_SUM    KS_SUM  KWKS_SUM
rouge1      0.297667  0.263120  0.300901  0.210458
rouge2      0.115736  0.094893  0.117795  0.078380
rougeL      0.191519  0.187298  0.220628  0.161458
rougeLsum   0.282497  0.245502  0.266103  0.194101